In [ ]:
!pip install scikit-learn==1.0.1
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy
!pip install kfp numpy

import pandas as pd 
import pickle
import tensorflow as tf
import numpy as np 
import time
import os

from kfp import components

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def read_file() -> None:
    import os
    import pandas as pd
    import numpy as np
    from minio import Minio
    from scipy.special import boxcox
    from sklearn.model_selection import train_test_split
    import boto3
    import json
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine, text
    import datetime
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    #Dictionary to save mean, sd, and, encoder
    # preprocess_df = {'version':1}
    
    #Perform normalization
    def zscore_normalization(df, name):
        mean = df[name].mean()
        sd = df[name].std()
        df[name] = (df[name] - mean) / sd
        # preprocess_df[name] = (mean, sd)
        
    #Encode text 
    def encode_text(df, name):
        from sklearn.preprocessing import OrdinalEncoder
        enc = OrdinalEncoder()
        data = enc.fit_transform(df[name].values.reshape(-1,1))
        df[name] = data.flatten()
        # preprocess_df[name + '_encoder'] = pickle.dumps(enc)
        
    #Data preprocessing
    def preprocess(df):
        df.columns = [
            'duration',
            'protocol_type',
            'service',
            'flag',
            'src_bytes',
            'dst_bytes',
            'land',
            'wrong_fragment',
            'urgent',
            'hot',
            'num_failed_logins',
            'logged_in',
            'num_compromised',
            'root_shell',
            'su_attempted',
            'num_root',
            'num_file_creations',
            'num_shells',
            'num_access_files',
            'num_outbound_cmds',
            'is_host_login',
            'is_guest_login',
            'count',
            'srv_count',
            'serror_rate',
            'srv_serror_rate',
            'rerror_rate',
            'srv_rerror_rate',
            'same_srv_rate',
            'diff_srv_rate',
            'srv_diff_host_rate',
            'dst_host_count',
            'dst_host_srv_count',
            'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate',
            'dst_host_same_src_port_rate',
            'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate',
            'dst_host_srv_serror_rate',
            'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate',
            'outcome'
        ]
        for col in df.columns:
            t = (df[col].dtype)
            if col != 'outcome':
                if (t == int or t == float):
                    zscore_normalization(df, col)
               
                else:
                    df[col] = df[col].astype(str)
                    encode_text(df, col)
                            
        for col in df.columns:
            if len(df[col].unique()) == 1:
                df.drop(col, inplace=True,axis=1)

        df.loc[df['outcome'] != "normal.", 'outcome']  = 1
        df.loc[df['outcome'] == "normal.", 'outcome']  = 0
        encode_text(df, "outcome")
        correlation = df.corrwith(df["outcome"])
      
        
        row = 0 
        for num in correlation:
            if num >= -0.05 and num <= 0.05:
                df.drop(df.columns[row], axis=1, inplace=True)
                row += 1
        return df
    
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    
    # Connect to PostgreSQL
    engine = create_engine(f'postgresql+psycopg2://{db_details["user"]}:{db_details["password"]}@{db_details["host"]}:{db_details["port"]}/{db_details["dbname"]}')
    
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()


    batch_size = 10000
    fetch_query = "SELECT * FROM intrusion_data where outcome != 2;"
            
    try:
        df = pd.DataFrame()

        df = pd.read_sql(fetch_query, conn)

    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None
        
    print(df.columns)
    
    df = df.drop(columns=['timestamp','uid'])
    df = preprocess(df)
    X = df.drop(columns=['outcome'])
    y = df['outcome']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    bucket_name="intrusion-pipeline"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    
    folder_path = './tmp/intrusion'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
        

    df.to_csv("./tmp/intrusion/intrusion_data.csv")
    s3_client.upload_file("./tmp/intrusion/intrusion_data.csv", bucket_name, "intrusion_dataset.csv")
    np.save("./tmp/intrusion/X_train.npy",X_train)
    s3_client.upload_file("./tmp/intrusion/X_train.npy", bucket_name, "X_train.npy")
    np.save("./tmp/intrusion/y_train.npy",y_train)
    s3_client.upload_file("./tmp/intrusion/y_train.npy", bucket_name, "y_train.npy")
    np.save("./tmp/intrusion/X_test.npy",X_test)
    s3_client.upload_file("./tmp/intrusion/X_test.npy", bucket_name, "X_test.npy")
    np.save("./tmp/intrusion/y_test.npy",y_test)
    s3_client.upload_file("./tmp/intrusion/y_test.npy", bucket_name, "y_test.npy")

In [ ]:
def train_op() -> None:
    import pickle
    import pandas as pd
    import numpy as np
    import json
    import os
    import time
    import tensorflow as tf
    import boto3
    from minio import Minio
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sqlalchemy import create_engine
    from sqlalchemy import create_engine, Table, Column, Float, Integer, String, MetaData, ARRAY
    from sqlalchemy import select, desc, insert, text
    from io import BytesIO
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    bucket_name="intrusion-pipeline"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    
    
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    response = s3_client.get_object(Bucket=bucket_name, Key="X_train.npy")
    data = response['Body'].read()
    X_train = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="y_train.npy")
    data = response['Body'].read()
    y_train = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="X_test.npy")
    data = response['Body'].read()
    X_test = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="y_test.npy")
    data = response['Body'].read()
    y_test = np.load(BytesIO(data))
    
    
    # Define dataframe to store model metrics
    metrics = pd.DataFrame(columns=["Version", "Model", "Accuracy", "F1", "Precision", "Recall", "Train_Time", "Test_Time"])
    models_path = './tmp/intrusion/models'
    
    if not os.path.exists(models_path):
        os.makedirs(models_path)
        print(f"Folder '{models_path}' created successfully.")
    else:
        print(f"Folder '{models_path}' already exists.")
        
    # version = 1
    # folder_path = f"version{version}"
    

    
    
    #Logistic Regression
    start_train = time.time()
    lrc = LogisticRegression(random_state=0, max_iter=1000)
    lrc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    ypredlr = lrc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, ypredlr)
    f1 = f1_score(y_test, ypredlr)
    precision = precision_score(y_test, ypredlr)
    recall = recall_score(y_test, ypredlr)
    metrics.loc[len(metrics.index)] = [version,'lrc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    # with open('./tmp/intrusion/models/lrc.pkl', 'wb') as f:
    #     pickle.dump(lrc, f)
    # s3_client.upload_file("tmp/intrusion/models/lrc.pkl", bucket_name, f"{folder_path}/lrc/model.pkl")
    
    
    #Random Forest Classifier
    start_train = time.time()
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred2=rfc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, y_pred2)
    f1 = f1_score(y_test, y_pred2)
    precision = precision_score(y_test, y_pred2)
    recall = recall_score(y_test, y_pred2)
    metrics.loc[len(metrics.index)] = [version, 'rfc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    # with open('./tmp/intrusion/models/rfc.pkl', 'wb') as f:
    #     pickle.dump(rfc, f)
    # s3_client.upload_file("tmp/intrusion/models/rfc.pkl", bucket_name, f"{folder_path}/rfc/model.pkl")
    
    
    #Decision Tree
    start_train = time.time()
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred3=dtc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred3)
    f1 = f1_score(y_test, y_pred3)
    precision = precision_score(y_test, y_pred3)
    recall = recall_score(y_test, y_pred3)
    metrics.loc[len(metrics.index)] = [version, 'dtc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    # with open('./tmp/intrusion/models/dtc.pkl', 'wb') as f:
    #     pickle.dump(dtc, f)
    # s3_client.upload_file("tmp/intrusion/models/dtc.pkl", bucket_name, f"{folder_path}/dtc/model.pkl")
    
    
    
    #Support Vector Machine
    start_train = time.time()
    svc = SVC()
    svc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred4=svc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred4)
    f1 = f1_score(y_test,y_pred4)
    precision = precision_score(y_test, y_pred4)
    recall = recall_score(y_test, y_pred4)
    metrics.loc[len(metrics.index)] = [version, 'svc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    # with open('./tmp/intrusion/models/svc.pkl', 'wb') as f:
    #     pickle.dump(svc, f)
    # s3_client.upload_file("tmp/intrusion/models/svc.pkl", bucket_name, f"{folder_path}/svc/model.pkl")
    
    
    
    #Gradient Boost
    start_train = time.time()
    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred5=gbc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred5)
    f1 = f1_score(y_test, y_pred5)
    precision = precision_score(y_test, y_pred5)
    recall = (recall_score(y_test, y_pred5))
    metrics.loc[len(metrics.index)] = [version, 'gbc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    # with open('./tmp/intrusion/models/gbc.pkl', 'wb') as f:
    #     pickle.dump(gbc, f)
    # s3_client.upload_file("tmp/intrusion/models/gbc.pkl", bucket_name, f"{folder_path}/gbc/model.pkl")
    
    
    #Gaussian Naive Bayes
    start_train = time.time()
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred6=gnb.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred6)
    f1 = f1_score(y_test, y_pred6)
    precision = precision_score(y_test,y_pred6)
    recall = recall_score(y_test, y_pred6)
    metrics.loc[len(metrics.index)] = [version, 'gnb', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]  
    # with open('./tmp/intrusion/models/gnb.pkl', 'wb') as f:
    #     pickle.dump(gnb, f)      
    # s3_client.upload_file("tmp/intrusion/models/gnb.pkl", bucket_name, f"{folder_path}/gnb/model.pkl")
    
    
    
    #Artificial Neural Network
    input_shape = [X_train.shape[1]]
    start_train = time.time()
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    model.build()
    model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=256, epochs=25)
    end_train=time.time()
    start_test = time.time()
    y_pred7 = model.predict(X_test)
    y_pred7 = (y_pred7 > 0.5).astype(np.int32)
    end_test = time.time()
    print(y_pred7)
    accuracy = accuracy_score(y_test,y_pred7)
    f1 = f1_score(y_test, y_pred7)
    precision = precision_score(y_test,y_pred7)
    recall = recall_score(y_test, y_pred7)
    # accuracy = history.history['accuracy'][11]
    metrics.loc[len(metrics.index)] = [version, 'ann', accuracy, f1, precision, recall, end_test-start_test, 0]
    # with open('./tmp/intrusion/models/ann.pkl', 'wb') as f:
    #     pickle.dump(model, f)
    # s3_client.upload_file("tmp/intrusion/models/ann.pkl", bucket_name, f"{folder_path}/ann/model.pkl")
    
    print(metrics)

#     db_details = {
#         'dbname': db,
#         'user': user,
#         'password': pswd,
#         'host': host,
#         'port': port
#     }
        
#     insert_query = """
#         INSERT INTO model_metrics (name, version, URI, in_use, accuracy, f1, precision, recall, train_time, test_time)
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#         ON CONFLICT (name, version) DO NOTHING;
#     """
#     try:
#         conn = psycopg2.connect(**db_details)
#         cursor = conn.cursor()
#         print("Connected to PostgreSQL successfully.")

#         # Iterate through DataFrame rows and insert into the table
#         for index, row in metrics.iterrows():
#             cursor.execute(insert_query, (
#                 row['Model'], 
#                 row['Version'], 
#                 f"s3://mlpipelineews/version{version}/{row['Model']}/model.pkl", 
#                 False, 
#                 row['Accuracy'], 
#                 row['F1'], 
#                 row['Precision'], 
#                 row['Recall'], 
#                 row['Train_Time'], 
#                 row['Test_Time']
#             ))
    
#         conn.commit()
#         print("Data inserted successfully.")

#         cursor.close()
#         conn.close()
#         print("PostgreSQL connection closed.")
#     except Exception as e:
#         print(f"Failed to connect to PostgreSQL or insert data: {e}")
    

In [ ]:
import kfp
from kfp import dsl
from kfp import components

read_csv_op = components.func_to_container_op(func=read_file, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas','scikit-learn==1.0.1', 'kfp', 'numpy', 'minio'])

train_op = components.func_to_container_op(func=train_op, output_component_file='train.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn==1.0.1','numpy','minio', 'tensorflow'])

read_data_op = kfp.components.load_component_from_file('preprocess.yaml')
train_op = kfp.components.load_component_from_file('train.yaml')


def ml_pipeline():
    preprocess = read_csv_op()
    train = train_op().after(preprocess)

# Compile the pipeline
kfp.compiler.Compiler().compile(ml_pipeline, 'intrusion_pipeline.yaml')